# Streamlit App UI Example

This notebook demonstrates an app interface that runs alongside the dashboard.

In [ ]:
# %load_ext autoreload
# %autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.resolve()))

from trulens_eval.keys import check_keys

check_keys(
    "OPENAI_API_KEY"
)

from trulens_eval import Tru

tru = Tru()
tru.reset_database()
tru.start_dashboard(
    force = True,
    _dev=Path().cwd().parent.parent.resolve()
)

## App loader

To be able to create a new session or "conversation", we need to be able to
reset the langchain app to its initial state. For this purpose, we require the
callable that produces a new chain that is configured for the start of the
conversation. Things like memory or other stateful aspects of the chain should
be at their initial values. Because of this, we need to construct it fully
inside the required callable.

**NOTE**: As another requirement of serialization, all of the relevant imports
that this function needs need to be included in its definition locally instead
of as global imports.

**WARNING**: This function will be naively serialized using `dill` and will fail a
size check if this serialization is too big. Because of this, you cannot rely on
large global objects in the function's closure and need to load them as part of
the execution of the function instead.

**WARNING**: This function needs to return a new instance of the app independent
of any others produced earlier. That is, you cannot take an existing or
pre-loaded app, clear its memory, and return it. As part of the dashboard,
multiple instances of an app need to operate at the same time without
interference in their states.

### Langchain example

In [ ]:
def load_langchain_app():
    # All relevant imports must be inside this function.

    from langchain.llms import OpenAI
    from langchain.chains import ConversationChain
    from langchain.memory import ConversationSummaryBufferMemory

    llm = OpenAI(temperature=0.9, max_tokens=128)

    # Conversation memory.
    memory = ConversationSummaryBufferMemory(
        k=4,
        max_token_limit=64,
        llm=llm,
    )

    # Conversational app puts it all together.
    app = ConversationChain(
        llm=llm,
        memory=memory
    )

    return app 

app1 = load_langchain_app()

tru_app1 = tru.Chain(
    app1,
    app_id='langchain_app',
    initial_app_loader=load_langchain_app
)

### Llama_index example

In [ ]:
def load_llamaindex_app():
    from llama_index import VectorStoreIndex, SimpleWebPageReader

    # Note that this is not an efficient way to create the app as it downloads
    # the web page every time the app needs to create a new session. Saving the
    # site locally would be a better idea.
    documents = SimpleWebPageReader(
        html_to_text=True
    ).load_data(["http://paulgraham.com/worked.html"])
    index = VectorStoreIndex.from_documents(documents)

    query_engine = index.as_query_engine()

    return query_engine

app2 = load_llamaindex_app()
tru_app2 = tru.Llama(
    app2,
    app_id="llamaindex_app",
    initial_app_loader=load_llamaindex_app
)

### Verification

You can get a list of apps that include the `initial_app_loader` with the following utility method.

In [ ]:
from trulens_eval.schema import AppDefinition

for app_json in AppDefinition.get_loadable_apps():
    print(app_json['app_id'])